# 01 — Data audit & EDA

## Objective
Assess whether the selected dataset is suitable for a Marketing Mix Model (MMM) and define
the exact setup to be used in the modeling phase.

## Inputs
- Raw multi-brand marketing and purchase dataset
- Problem definition and KPI decisions from `00_problem_context.ipynb`

## Output
- Explicit decision on dataset suitability
- Selected time series (organisation + territory)
- Agreed target, inputs, and time granularity


# 01 — Data audit & EDA

## Objective
Load and audit the dataset to assess whether it is suitable for a Marketing Mix Model (MMM).

## What “suitable” means here
To be useful for MMM, the dataset must contain:
- A time index (daily or weekly)
- A business KPI aggregated by time (e.g. purchases/orders)
- Marketing inputs by channel (ideally spend)
- Enough history to validate out-of-sample (temporal split)

## Output of this notebook
A clear decision:
- ✅ Can be used as-is
- ⚠️ Can be used with adjustments
- ❌ Should be discarded


# dataset source
https://figshare.com/articles/dataset/Multi-Region_Marketing_Mix_Modeling_MMM_Dataset_for_Several_eCommerce_Brands/25314841/3

In [1]:
# Librerías

import pandas as pd

FILE_PATH = "../data/raw/mmm_raw_dataset.csv"  

df = pd.read_csv(FILE_PATH)
df.head()

,MMM_TIMESERIES_ID,ORGANISATION_ID,ORGANISATION_VERTICAL,ORGANISATION_SUBVERTICAL,ORGANISATION_MARKETING_SOURCES,ORGANISATION_PRIMARY_TERRITORY_NAME,TERRITORY_NAME,DATE_DAY,CURRENCY_CODE,FIRST_PURCHASES,...,META_FACEBOOK_IMPRESSIONS,META_INSTAGRAM_IMPRESSIONS,META_OTHER_IMPRESSIONS,TIKTOK_IMPRESSIONS,DIRECT_CLICKS,BRANDED_SEARCH_CLICKS,ORGANIC_SEARCH_CLICKS,EMAIL_CLICKS,REFERRAL_CLICKS,ALL_OTHER_CLICKS
0,596eef7c71f933d820d0e485935d0e8f,04769dac8b828ec7a85676d9e2bffe6f,Beauty & Fitness,Hair Care,"Google, Meta",US,All Territories,2022-07-29,USD,22,...,18997.0,NaN,NaN,NaN,139.0,NaN,300.0,1.0,61.0,40.0
1,596eef7c71f933d820d0e485935d0e8f,04769dac8b828ec7a85676d9e2bffe6f,Beauty & Fitness,Hair Care,"Google, Meta",US,All Territories,2022-07-30,USD,14,...,20188.0,NaN,NaN,NaN,209.0,NaN,442.0,8.0,110.0,62.0
2,596eef7c71f933d820d0e485935d0e8f,04769dac8b828ec7a85676d9e2bffe6f,Beauty & Fitness,Hair Care,"Google, Meta",US,All Territories,2022-07-31,USD,31,...,24718.0,NaN,NaN,NaN,262.0,NaN,427.0,631.0,108.0,65.0
3,596eef7c71f933d820d0e485935d0e8f,04769dac8b828ec7a85676d9e2bffe6f,Beauty & Fitness,Hair Care,"Google, Meta",US,All Territories,2022-08-01,USD,18,...,25076.0,NaN,NaN,NaN,247.0,NaN,400.0,117.0,125.0,68.0
4,596eef7c71f933d820d0e485935d0e8f,04769dac8b828ec7a85676d9e2bffe6f,Beauty & Fitness,Hair Care,"Google, Meta",US,All Territories,2022-08-02,USD,23,...,22688.0,NaN,NaN,NaN,255.0,NaN,425.0,37.0,146.0,65.0


## Dataset quick overview

We start by checking:
- number of rows and columns
- column types
- missingness patterns (especially in spend columns)

In [2]:
df.shape, df.dtypes.head(10)


((132759, 50),
 MMM_TIMESERIES_ID                      object
 ORGANISATION_ID                        object
 ORGANISATION_VERTICAL                  object
 ORGANISATION_SUBVERTICAL               object
 ORGANISATION_MARKETING_SOURCES         object
 ORGANISATION_PRIMARY_TERRITORY_NAME    object
 TERRITORY_NAME                         object
 DATE_DAY                               object
 CURRENCY_CODE                          object
 FIRST_PURCHASES                         int64
 dtype: object)

In [3]:
df.info()
missing_rate = df.isna().mean().sort_values(ascending=False)
missing_rate.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132759 entries, 0 to 132758
Data columns (total 50 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   MMM_TIMESERIES_ID                    132759 non-null  object 
 1   ORGANISATION_ID                      132759 non-null  object 
 2   ORGANISATION_VERTICAL                124649 non-null  object 
 3   ORGANISATION_SUBVERTICAL             124649 non-null  object 
 4   ORGANISATION_MARKETING_SOURCES       132759 non-null  object 
 5   ORGANISATION_PRIMARY_TERRITORY_NAME  132759 non-null  object 
 6   TERRITORY_NAME                       132759 non-null  object 
 7   DATE_DAY                             132759 non-null  object 
 8   CURRENCY_CODE                        132759 non-null  object 
 9   FIRST_PURCHASES                      132759 non-null  int64  
 10  FIRST_PURCHASES_UNITS                132759 non-null  int64  
 11  FIRST_PURCHAS

TIKTOK_CLICKS                  0.973923
TIKTOK_SPEND                   0.973923
TIKTOK_IMPRESSIONS             0.973923
GOOGLE_VIDEO_SPEND             0.942543
GOOGLE_VIDEO_IMPRESSIONS       0.942543
GOOGLE_VIDEO_CLICKS            0.942543
GOOGLE_DISPLAY_SPEND           0.858797
GOOGLE_DISPLAY_CLICKS          0.858797
GOOGLE_DISPLAY_IMPRESSIONS     0.858797
META_OTHER_CLICKS              0.822649
META_OTHER_IMPRESSIONS         0.822649
META_OTHER_SPEND               0.822649
META_INSTAGRAM_SPEND           0.798002
META_INSTAGRAM_CLICKS          0.798002
META_INSTAGRAM_IMPRESSIONS     0.798002
GOOGLE_PMAX_CLICKS             0.539572
GOOGLE_PMAX_SPEND              0.539572
GOOGLE_PMAX_IMPRESSIONS        0.539572
GOOGLE_SHOPPING_IMPRESSIONS    0.405087
GOOGLE_SHOPPING_CLICKS         0.405087
dtype: float64

## Missing values interpretation and decision

Several marketing spend columns show high missing rates (e.g. TikTok, Google Video, Display).

Interpretation:
- Missing values in spend columns are interpreted as periods where the channel was inactive.
- This is expected in multi-brand, multi-channel marketing data.

Decision:
- In preprocessing, missing values in selected paid media spend columns will be replaced with 0.
- This replacement will be done after selecting a single time series and aggregating to weekly frequency.

Rationale:
- Treating missing spend as zero is standard practice in MMM.
- Dropping these rows would incorrectly remove valid observations where other channels were active.


In [4]:
spend_cols = [c for c in df.columns if c.endswith("_SPEND")]
click_cols = [c for c in df.columns if c.endswith("_CLICKS")]
imp_cols = [c for c in df.columns if c.endswith("_IMPRESSIONS")]

len(spend_cols), len(click_cols), len(imp_cols)


(9, 15, 9)

## Marketing input selection decision

The dataset contains:
- 9 paid media spend channels
- 15 click-based metrics
- 9 impression-based metrics

Decision for the baseline MMM:
- Use only spend-based channels as marketing inputs.
- Exclude clicks and impressions from the initial model.

Rationale:
- Spend is the most standard and interpretable MMM input.
- Click and impression metrics are derived signals and may introduce redundancy.
- A spend-only baseline reduces multicollinearity and keeps the first model simple.


In [5]:
target_col = "ALL_PURCHASES"
date_col = "DATE_DAY"

df[date_col] = pd.to_datetime(df[date_col])
df[[date_col, target_col]].head()


,DATE_DAY,ALL_PURCHASES
0,2022-07-29,27
1,2022-07-30,17
2,2022-07-31,39
3,2022-08-01,22
4,2022-08-02,28


## Target KPI definition

The target KPI selected for the MMM is:

- Name: ALL_PURCHASES
- Definition: Total number of purchases completed in a given period.
- Unit: Count
- Granularity: Daily (to be aggregated to weekly in the modeling phase)

Rationale:
- Purchase counts are stable and interpretable.
- They align well with the concept of conversions in MMM.
- Revenue-based KPIs can be explored in later iterations.


In [6]:
df[["ORGANISATION_ID", "TERRITORY_NAME"]].drop_duplicates().shape


(143, 2)

## Multi-series structure

The dataset contains 143 distinct (ORGANISATION_ID, TERRITORY_NAME) time series.

Decision:
- For the from-scratch MMM, we will focus on a single time series.
- This implies selecting one organisation and one territory.

Rationale:
- Avoid mixing heterogeneous businesses.
- Keep modeling assumptions interpretable.
- Enable clear diagnostics and validation.


In [7]:
spend_cols = [c for c in df.columns if c.endswith("_SPEND")]

series_activity = (
    df.assign(total_spend=df[spend_cols].fillna(0).sum(axis=1))
      .groupby(["ORGANISATION_ID", "TERRITORY_NAME"], as_index=False)
      .agg(
          n_days=(date_col, "count"),
          total_spend=("total_spend", "sum"),
          total_purchases=(target_col, "sum"),
      )
)

series_activity.sort_values(
    ["n_days", "total_spend"],
    ascending=False
).head(10)


,ORGANISATION_ID,TERRITORY_NAME,n_days,total_spend,total_purchases
66,72a86a208d24d68b80be0e44a8a4872d,All Territories,1751,6.747559e+05,55405
83,882ce7e286d66facc66518783e2192c7,All Territories,1711,4.051800e+06,630667
88,882ce7e286d66facc66518783e2192c7,UK,1711,2.996593e+06,356719
86,882ce7e286d66facc66518783e2192c7,Germany,1711,7.857252e+05,75946
87,882ce7e286d66facc66518783e2192c7,Spain,1705,3.456558e+05,33059
117,bfb6f6a326141ed6a751fc83ba836984,All Territories,1654,2.352877e+06,495035
64,7059e30b528ed5f14ee9921de13248e5,All Territories,1642,2.076258e+06,169714
103,ba773ebd7ec0a08f1d042187d086ccb4,All Territories,1610,7.687312e+06,1059108
108,ba773ebd7ec0a08f1d042187d086ccb4,Germany,1610,7.587962e+05,73034
115,ba773ebd7ec0a08f1d042187d086ccb4,US,1600,2.344113e+06,270056


## Selected time series for baseline MMM

After evaluating activity across all available series, we select the following
time series for the initial MMM:

- ORGANISATION_ID: 72a86a208d24d68b80be0e44a8a4872d
- TERRITORY_NAME: All Territories

Rationale:
- Long historical coverage (~1750 days)
- Meaningful but not extreme marketing spend
- Sufficient purchase volume
- Single, coherent time series without overlapping regional breakdowns

This series provides a good balance between realism and interpretability for
learning MMM from scratch.


In [8]:
ORG_ID = "72a86a208d24d68b80be0e44a8a4872d"
TERRITORY = "All Territories"

df_one = df[
    (df["ORGANISATION_ID"] == ORG_ID) &
    (df["TERRITORY_NAME"] == TERRITORY)
].copy()

df_one.shape, df_one[date_col].min(), df_one[date_col].max()


((1751, 50),
 Timestamp('2019-08-16 00:00:00'),
 Timestamp('2024-05-31 00:00:00'))

## Time granularity decision

The raw data is available at daily frequency.

Decision:
- For the baseline MMM, we will aggregate the data to weekly frequency.

Rationale:
- Weekly aggregation reduces daily noise.
- It smooths sparse channel activity.
- It aligns with common MMM practices and improves model stability.


## Final dataset decision

✅ This dataset is suitable for MMM modeling with the following setup:

- Single time series:
  - ORGANISATION_ID: 72a86a208d24d68b80be0e44a8a4872d
  - TERRITORY_NAME: All Territories
- Target KPI: ALL_PURCHASES
- Marketing inputs: paid media spend columns
- Time granularity: daily → weekly
- Missing spend values interpreted as inactive channels (to be set to zero)

Key look-ahead:
- Feature engineering (adstock, saturation) will be applied after aggregation.
- Validation will be done using a temporal holdout.


## Final decision and rationale

### Decision
The dataset is suitable for MMM modeling with controlled adjustments.

### Rationale
- The dataset contains a long and stable historical time series.
- Paid media spend is available for multiple channels with realistic sparsity patterns.
- Purchase counts provide a stable and interpretable target KPI.
- Selecting a single organisation/territory avoids heterogeneity issues.
- Weekly aggregation will reduce noise and improve model stability.

This setup provides a strong and interpretable foundation for a from-scratch MMM.
